# Model Predictive Control (MPC) for Autonomous Vehicles

## Introduction

Welcome to this interactive tutorial on Model Predictive Control (MPC) for autonomous vehicle trajectory tracking! This notebook demonstrates how to implement and visualize a Model Predictive Controller for a simple vehicle model to follow predefined trajectories.

Model Predictive Control is an advanced control technique that uses a model of the system to predict future states and optimize control inputs over a receding horizon. It's widely used in autonomous vehicles, robotics, and industrial processes due to its ability to handle constraints and optimize performance.

## What You'll Learn

- The fundamentals of Model Predictive Control
- How to formulate an optimization problem for vehicle control
- How to implement MPC using Python and CVXPY
- How different parameters affect controller performance
- Vehicle kinematics and motion planning basics

## How to Use This Notebook

### Getting Started

1. Start by running the first two cells to install dependencies and import necessary libraries
2. Run the subsequent cells to load utility functions and set up vehicle parameters
3. The final two cells run a simulation with a predefined trajectory

### Available Trajectories

This notebook includes several predefined trajectories:

- **Straight**: A simple straight line
- **Wavy**: A slightly curved path
- **Spline**: A complex curved path
- **Circular**: A circular track
- **Eternity**: A figure-eight path
- **Slalom**: A zigzag path
- **Custom**: A custom path you can modify

### Visualizations

The notebook provides three types of visualizations:

1. **Static overview**: Shows the entire trajectory with reference path and actual path
2. **Animation**: Displays the vehicle's motion and MPC predictions frame by frame
3. **Performance plots**: Shows speed and steering profiles over time

### Simulation Options

You have two simulation options:
- Running in real time for better visualization, but it's slower
- Running in backgroun and showing the final animations. It's faster and you can go back and forth after the animation is made.

NOTE: for better usage, you can use the project outside Colab: https://github.com/SaeedRahmani/mpc__iv_course

### Modifying the MPC Controller

To experiment with the MPC controller itself, look at the following:

- Cell: Vehicle parameters and state definitions
- Cell: MPC parameters and cost function implementation
- The `linear_mpc_control()` function contains the core optimization problem
- You can also creat your own trajectory by modifying the related cell

## Key Components

### Vehicle Model

The notebook uses a simple bicycle model with states:

- x, y: Position
- v: Speed
- yaw: Orientation

And control inputs:

- a: Acceleration/deceleration
- δ: Steering angle

### MPC Implementation (Optional)

The MPC controller:

1. Predicts the future states of the vehicle
2. Optimizes control inputs to minimize tracking error and control effort
3. Applies the first optimal control input
4. Repeats the process at the next time step

## Technical Notes (optional)

- The optimization problem is solved using CVXPY with the CLARABEL solver
- The simulation uses a linearized model for computational efficiency
- The controller includes constraints on speed, acceleration, and steering

## Exercises for Students

1. Try different trajectories and speeds and observe how the controller performs
2. Modify the MPC cost function weights (Q, R, Rd) and observe changes in behavior
3. Experiment with different prediction horizons (T) and sample times (DT)
4. Add obstacles to the environment and modify the cost function to avoid them
5. Implement your own custom trajectory by modifying the `custom_path()` function

Enjoy exploring Model Predictive Control for autonomous vehicles!

# Imports and Installs (only once)

In [ ]:
# # Install required packages (If you see anything is missing, comment out the following two lines and run the cell)
!pip install --quiet numpy scipy matplotlib cvxpy ipympl ipywidgets

In [ ]:
# Import standard libraries
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import cvxpy

# For interactive plotting in the notebook
%matplotlib inline

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from matplotlib import rc
rc('animation', html='jshtml')    # tells Matplotlib to emit JS+<canvas>

## Utility Functions (Do not touch)

In [ ]:
# Angle utility functions from utils/angle.py
import numpy as np
from scipy.spatial.transform import Rotation as Rot

def rot_mat_2d(angle):
    """Create 2D rotation matrix from an angle"""
    return Rot.from_euler('z', angle).as_matrix()[0:2, 0:2]

def angle_mod(x, zero_2_2pi=False, degree=False):
    """
    Angle modulo operation
    Default angle modulo range is [-pi, pi)
    """
    if isinstance(x, float):
        is_float = True
    else:
        is_float = False

    x = np.asarray(x).flatten()
    if degree:
        x = np.deg2rad(x)

    if zero_2_2pi:
        mod_angle = x % (2 * np.pi)
    else:
        mod_angle = (x + np.pi) % (2 * np.pi) - np.pi

    if degree:
        mod_angle = np.rad2deg(mod_angle)

    if is_float:
        return mod_angle.item()
    else:
        return mod_angle

def pi_2_pi(angle):
    """Convert angle to range [-pi, pi)"""
    return angle_mod(angle)

# Cubic spline planner implementation from utils/cubic_spline_planner.py
class CubicSpline1D:
    """
    1D Cubic Spline class for interpolation
    """
    def __init__(self, x, y):
        h = np.diff(x)
        if np.any(h < 0):
            raise ValueError("x coordinates must be sorted in ascending order")

        self.a, self.b, self.c, self.d = [], [], [], []
        self.x = x
        self.y = y
        self.nx = len(x)  # dimension of x

        # calc coefficient a
        self.a = [iy for iy in y]

        # calc coefficient c
        A = self.__calc_A(h)
        B = self.__calc_B(h, self.a)
        self.c = np.linalg.solve(A, B)

        # calc spline coefficient b and d
        for i in range(self.nx - 1):
            d = (self.c[i + 1] - self.c[i]) / (3.0 * h[i])
            b = 1.0 / h[i] * (self.a[i + 1] - self.a[i]) \
                - h[i] / 3.0 * (2.0 * self.c[i] + self.c[i + 1])
            self.d.append(d)
            self.b.append(b)

    def calc_position(self, x):
        """Calculate position at x"""
        if x < self.x[0]:
            return None
        elif x > self.x[-1]:
            return None

        i = self.__search_index(x)
        dx = x - self.x[i]
        position = self.a[i] + self.b[i] * dx + \
            self.c[i] * dx ** 2.0 + self.d[i] * dx ** 3.0

        return position

    def calc_first_derivative(self, x):
        """Calculate first derivative at x"""
        if x < self.x[0]:
            return None
        elif x > self.x[-1]:
            return None

        i = self.__search_index(x)
        dx = x - self.x[i]
        dy = self.b[i] + 2.0 * self.c[i] * dx + 3.0 * self.d[i] * dx ** 2.0
        return dy

    def calc_second_derivative(self, x):
        """Calculate second derivative at x"""
        if x < self.x[0]:
            return None
        elif x > self.x[-1]:
            return None

        i = self.__search_index(x)
        dx = x - self.x[i]
        ddy = 2.0 * self.c[i] + 6.0 * self.d[i] * dx
        return ddy

    def calc_third_derivative(self, x):
        """Calculate third derivative at x"""
        if x < self.x[0]:
            return None
        elif x > self.x[-1]:
            return None

        i = self.__search_index(x)
        dddy = 6.0 * self.d[i]
        return dddy

    def __search_index(self, x):
        """Search data segment index"""
        import bisect
        return bisect.bisect(self.x, x) - 1

    def __calc_A(self, h):
        """Calculate matrix A for spline coefficient c"""
        A = np.zeros((self.nx, self.nx))
        A[0, 0] = 1.0
        for i in range(self.nx - 1):
            if i != (self.nx - 2):
                A[i + 1, i + 1] = 2.0 * (h[i] + h[i + 1])
            A[i + 1, i] = h[i]
            A[i, i + 1] = h[i]

        A[0, 1] = 0.0
        A[self.nx - 1, self.nx - 2] = 0.0
        A[self.nx - 1, self.nx - 1] = 1.0
        return A

    def __calc_B(self, h, a):
        """Calculate matrix B for spline coefficient c"""
        B = np.zeros(self.nx)
        for i in range(self.nx - 2):
            B[i + 1] = 3.0 * (a[i + 2] - a[i + 1]) / h[i + 1] - 3.0 * (a[i + 1] - a[i]) / h[i]
        return B


class CubicSpline2D:
    """
    2D Cubic Spline class for trajectory generation
    """
    def __init__(self, x, y):
        self.s = self.__calc_s(x, y)
        self.sx = CubicSpline1D(self.s, x)
        self.sy = CubicSpline1D(self.s, y)

    def __calc_s(self, x, y):
        dx = np.diff(x)
        dy = np.diff(y)
        self.ds = np.hypot(dx, dy)
        s = [0]
        s.extend(np.cumsum(self.ds))
        return s

    def calc_position(self, s):
        """Calculate position at s"""
        x = self.sx.calc_position(s)
        y = self.sy.calc_position(s)
        return x, y

    def calc_curvature(self, s):
        """Calculate curvature at s"""
        dx = self.sx.calc_first_derivative(s)
        ddx = self.sx.calc_second_derivative(s)
        dy = self.sy.calc_first_derivative(s)
        ddy = self.sy.calc_second_derivative(s)
        k = (ddy * dx - ddx * dy) / ((dx ** 2 + dy ** 2)**(3 / 2))
        return k

    def calc_yaw(self, s):
        """Calculate yaw angle at s"""
        dx = self.sx.calc_first_derivative(s)
        dy = self.sy.calc_first_derivative(s)
        yaw = math.atan2(dy, dx)
        return yaw


def calc_spline_course(x, y, ds=0.1):
    """
    Calculate spline course from waypoints
    Returns course position, yaw and curvature lists
    """
    sp = CubicSpline2D(x, y)
    s = list(np.arange(0, sp.s[-1], ds))

    rx, ry, ryaw, rk = [], [], [], []
    for i_s in s:
        ix, iy = sp.calc_position(i_s)
        rx.append(ix)
        ry.append(iy)
        ryaw.append(sp.calc_yaw(i_s))
        rk.append(sp.calc_curvature(i_s))

    return rx, ry, ryaw, rk, s

def plot_car(x, y, yaw, steer=0.0, cabcolor="-r", truckcolor="-k", ax=None):
    """
    Plot a car on a 2D surface

    Parameters:
    ----------
    x, y: position
    yaw: orientation
    steer: steering angle
    cabcolor, truckcolor: colors
    ax: matplotlib axes (optional)
    """
    # If ax is None, use current axes
    if ax is None:
        ax = plt.gca()

    outline = np.array([[-BACKTOWHEEL, (LENGTH - BACKTOWHEEL), (LENGTH - BACKTOWHEEL), -BACKTOWHEEL, -BACKTOWHEEL],
                        [WIDTH / 2, WIDTH / 2, - WIDTH / 2, -WIDTH / 2, WIDTH / 2]])

    fr_wheel = np.array([[WHEEL_LEN, -WHEEL_LEN, -WHEEL_LEN, WHEEL_LEN, WHEEL_LEN],
                         [-WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD]])

    rr_wheel = np.copy(fr_wheel)

    fl_wheel = np.copy(fr_wheel)
    fl_wheel[1, :] *= -1
    rl_wheel = np.copy(rr_wheel)
    rl_wheel[1, :] *= -1

    Rot1 = np.array([[math.cos(yaw), math.sin(yaw)],
                     [-math.sin(yaw), math.cos(yaw)]])
    Rot2 = np.array([[math.cos(steer), math.sin(steer)],
                     [-math.sin(steer), math.cos(steer)]])

    fr_wheel = (fr_wheel.T.dot(Rot2)).T
    fl_wheel = (fl_wheel.T.dot(Rot2)).T
    fr_wheel[0, :] += WB
    fl_wheel[0, :] += WB

    fr_wheel = (fr_wheel.T.dot(Rot1)).T
    fl_wheel = (fl_wheel.T.dot(Rot1)).T

    outline = (outline.T.dot(Rot1)).T
    rr_wheel = (rr_wheel.T.dot(Rot1)).T
    rl_wheel = (rl_wheel.T.dot(Rot1)).T

    outline[0, :] += x
    outline[1, :] += y
    fr_wheel[0, :] += x
    fr_wheel[1, :] += y
    rr_wheel[0, :] += x
    rr_wheel[1, :] += y
    fl_wheel[0, :] += x
    fl_wheel[1, :] += y
    rl_wheel[0, :] += x
    rl_wheel[1, :] += y

    ax.plot(np.array(outline[0, :]).flatten(),
             np.array(outline[1, :]).flatten(), truckcolor)
    ax.plot(np.array(fr_wheel[0, :]).flatten(),
             np.array(fr_wheel[1, :]).flatten(), truckcolor)
    ax.plot(np.array(rr_wheel[0, :]).flatten(),
             np.array(rr_wheel[1, :]).flatten(), truckcolor)
    ax.plot(np.array(fl_wheel[0, :]).flatten(),
             np.array(fl_wheel[1, :]).flatten(), truckcolor)
    ax.plot(np.array(rl_wheel[0, :]).flatten(),
             np.array(rl_wheel[1, :]).flatten(), truckcolor)
    ax.plot(x, y, "*")

def smooth_yaw(yaw):
    """
    Smooth yaw angle changes
    """
    for i in range(len(yaw) - 1):
        dyaw = yaw[i + 1] - yaw[i]

        while dyaw >= math.pi / 2.0:
            yaw[i + 1] -= math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

        while dyaw <= -math.pi / 2.0:
            yaw[i + 1] += math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

    return yaw

def calc_ref_trajectory(state, cx, cy, cyaw, ck, sp, dl, pind):
    """
    Calculate reference trajectory at current state
    """
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    ind, _ = calc_nearest_index(state, cx, cy, cyaw, pind)

    if pind >= ind:
        ind = pind

    xref[0, 0] = cx[ind]
    xref[1, 0] = cy[ind]
    xref[2, 0] = sp[ind]
    xref[3, 0] = cyaw[ind]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        travel += abs(state.v) * DT
        dind = int(round(travel / dl))

        if (ind + dind) < ncourse:
            xref[0, i] = cx[ind + dind]
            xref[1, i] = cy[ind + dind]
            xref[2, i] = sp[ind + dind]
            xref[3, i] = cyaw[ind + dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = cx[ncourse - 1]
            xref[1, i] = cy[ncourse - 1]
            xref[2, i] = sp[ncourse - 1]
            xref[3, i] = cyaw[ncourse - 1]
            dref[0, i] = 0.0

    return xref, ind, dref

def calc_speed_profile(cx, cy, cyaw, target_speed):
    """
    Calculate speed profile
    """
    speed_profile = [target_speed] * len(cx)
    direction = 1.0  # forward

    # Set stop point
    for i in range(len(cx) - 1):
        dx = cx[i + 1] - cx[i]
        dy = cy[i + 1] - cy[i]

        move_direction = math.atan2(dy, dx)

        if dx != 0.0 and dy != 0.0:
            dangle = abs(pi_2_pi(move_direction - cyaw[i]))
            if dangle >= math.pi / 4.0:
                direction = -1.0
            else:
                direction = 1.0

        if direction != 1.0:
            speed_profile[i] = - target_speed
        else:
            speed_profile[i] = target_speed

    speed_profile[-1] = 0.0

    return speed_profile

def check_goal(state, goal, tind, nind):
    """
    Check if vehicle reaches goal
    """
    # check goal
    dx = state.x - goal[0]
    dy = state.y - goal[1]
    d = math.hypot(dx, dy)

    isgoal = (d <= GOAL_DIS)

    if abs(tind - nind) >= 5:
        isgoal = False

    isstop = (abs(state.v) <= STOP_SPEED)

    if isgoal and isstop:
        return True

    return False

## Building the Road (Reference Trajectory)

In [ ]:
def straight_path():
    """Straight course path"""
    ax = [0.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0]
    ay = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    waypoints = list(zip(ax, ay))
    return waypoints

def wavy_path():
    """Slightly curved path"""
    ax = [0.0, -10.0, -20.0, -40.0, -50.0, -60.0, -70.0]
    ay = [0.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0]
    waypoints = list(zip(ax, ay))
    return waypoints

def spline_path():
    """Forward course path"""
    ax = [0.0, 60.0, 125.0, 50.0, 75.0, 30.0, -10.0]
    ay = [0.0, 0.0, 50.0, 65.0, 30.0, 50.0, -20.0]
    waypoints = list(zip(ax, ay))
    return waypoints

def circular_path():
    """Circular path"""
    radius = 30.0
    center_x, center_y = 50.0, 0.0

    # Generate points along a circle
    angles = np.linspace(0, 2*np.pi, 12)
    waypoints = [(center_x + radius * np.cos(angle),
                  center_y + radius * np.sin(angle)) for angle in angles]

    return waypoints

def figure_eight_path():
    """Figure-8 path"""
    radius = 30.0

    # Generate points for the figure-8
    angles = np.linspace(0, 2*np.pi, 16)
    waypoints = []

    for angle in angles:
        # Figure-8 parametric equations
        x = radius * np.cos(angle)
        y = radius/2 * np.sin(2*angle)
        waypoints.append((x, y))

    return waypoints

def slalom_path():
    """Slalom path"""
    waypoints = [
        (0.0, 0.0),
        (20.0, 10.0),
        (40.0, -10.0),
        (60.0, 10.0),
        (80.0, -10.0),
        (100.0, 0.0)
    ]
    return waypoints

def custom_path():
    """Custom path - modify this for your own trajectory"""
    waypoints = [
        (0.0, 0.0),    # Starting point
        (10.0, 10.0),  # Second point
        (20.0, 0.0),   # Third point
        (30.0, -10.0), # Fourth point
        (40.0, 0.0)    # Fifth point
    ]
    return waypoints

# Needed for GUI
def create_custom_trajectory(waypoints, dl=1.0):
    """
    Create a trajectory from user-defined waypoints.

    Parameters
    ----------
    waypoints : list of tuples
        List of (x, y) coordinates defining the waypoints of the trajectory
    dl : float, optional
        Distance between interpolated points, by default 1.0

    Returns
    -------
    tuple
        (cx, cy, cyaw, ck) - course x, y, yaw, and curvature lists
    """
    ax = [point[0] for point in waypoints]
    ay = [point[1] for point in waypoints]

    cx, cy, cyaw, ck, _ = calc_spline_course(ax, ay, ds=dl)

    return cx, cy, cyaw, ck

# Dictionary mapping trajectory names to functions
TRAJECTORIES = {
    "Straight": straight_path,
    "Wavy": wavy_path,
    "Spline": spline_path,
    "Circular": circular_path,
    "Eternity": figure_eight_path,
    "Slalom": slalom_path,
    "Custom": custom_path,
}

# Default trajectory to use
DEFAULT_TRAJECTORY = "Circular"

## Vehicle State and MPC Parameters
### For modifying the vehicle and MPC parameters


In [ ]:
# Vehicle model and state definitions from py

# State and input dimensions
NX = 4  # x = x, y, v, yaw
NU = 2  # a = [accel, steer]
T = 5   # horizon length

# MPC parameters (default values)
R = np.diag([0.01, 0.01])   # input cost matrix
Rd = np.diag([0.01, 1.0])   # input difference cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
Qf = Q  # state final matrix
GOAL_DIS = 1.5  # goal distance
STOP_SPEED = 0.5 / 3.6  # stop speed
MAX_TIME = 500.0  # max simulation time

# Iterative parameter
MAX_ITER = 3  # Max iteration
DU_TH = 0.1  # iteration finish param

TARGET_SPEED = 10.0 / 3.6  # [m/s] target speed
N_IND_SEARCH = 10  # Search index number

DT = 0.2  # [s] time tick

# Vehicle parameters
LENGTH = 4.5  # [m]
WIDTH = 2.0  # [m]
BACKTOWHEEL = 1.0  # [m]
WHEEL_LEN = 0.3  # [m]
WHEEL_WIDTH = 0.2  # [m]
TREAD = 0.7  # [m]
WB = 2.5  # [m]

MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 1.0  # maximum accel [m/ss]

show_animation = True


class State:
    """
    Vehicle state class
    """
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None

## MPC Core Functions
### For modifying the MPC cost function look into linear_mpc_control()

In [ ]:
# MPC core functions from py

def get_linear_model_matrix(v, phi, delta):
    """
    Create linearized model matrices
    """
    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)

    return A, B, C


def update_state(state, a, delta):
    """
    Update vehicle state with control inputs
    """
    # input check
    if delta >= MAX_STEER:
        delta = MAX_STEER
    elif delta <= -MAX_STEER:
        delta = -MAX_STEER

    state.x = state.x + state.v * math.cos(state.yaw) * DT
    state.y = state.y + state.v * math.sin(state.yaw) * DT
    state.yaw = state.yaw + state.v / WB * math.tan(delta) * DT
    state.v = state.v + a * DT

    if state.v > MAX_SPEED:
        state.v = MAX_SPEED
    elif state.v < MIN_SPEED:
        state.v = MIN_SPEED

    return state


def get_nparray_from_matrix(x):
    """
    Convert matrix to numpy array
    """
    return np.array(x).flatten()


def calc_nearest_index(state, cx, cy, cyaw, pind):
    """
    Calculate nearest point index to current state
    """
    dx = [state.x - icx for icx in cx[pind:(pind + N_IND_SEARCH)]]
    dy = [state.y - icy for icy in cy[pind:(pind + N_IND_SEARCH)]]

    d = [idx ** 2 + idy ** 2 for (idx, idy) in zip(dx, dy)]

    mind = min(d)

    ind = d.index(mind) + pind

    mind = math.sqrt(mind)

    dxl = cx[ind] - state.x
    dyl = cy[ind] - state.y

    angle = pi_2_pi(cyaw[ind] - math.atan2(dyl, dxl))
    if angle < 0:
        mind *= -1

    return ind, mind


def predict_motion(x0, oa, od, xref):
    """
    Predict vehicle motion with given control inputs
    """
    xbar = xref * 0.0
    for i, _ in enumerate(x0):
        xbar[i, 0] = x0[i]

    state = State(x=x0[0], y=x0[1], yaw=x0[3], v=x0[2])
    for (ai, di, i) in zip(oa, od, range(1, T + 1)):
        state = update_state(state, ai, di)
        xbar[0, i] = state.x
        xbar[1, i] = state.y
        xbar[2, i] = state.v
        xbar[3, i] = state.yaw

    return xbar


def iterative_linear_mpc_control(xref, x0, dref, oa, od):
    """
    MPC control with updating operational point iteratively
    """
    ox, oy, oyaw, ov = None, None, None, None

    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T

    for i in range(MAX_ITER):
        xbar = predict_motion(x0, oa, od, xref)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, x0, dref)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")

    return oa, od, ox, oy, oyaw, ov


def linear_mpc_control(xref, xbar, x0, dref):
    """
    Linear MPC control
    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """
    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(
            xbar[2, t], xbar[3, t], dref[0, t])
        constraints += [x[:, t + 1] == A @ x[:, t] + B @ u[:, t] + C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <=
                            MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == x0]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.CLARABEL, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = get_nparray_from_matrix(x.value[0, :])
        oy = get_nparray_from_matrix(x.value[1, :])
        ov = get_nparray_from_matrix(x.value[2, :])
        oyaw = get_nparray_from_matrix(x.value[3, :])
        oa = get_nparray_from_matrix(u.value[0, :])
        odelta = get_nparray_from_matrix(u.value[1, :])

    else:
        print("Error: Cannot solve .")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov

## Main Simulation Function

In [ ]:
def do_simulation(cx, cy, cyaw, ck, sp, dl, initial_state, show_animation=True):
    """
    Simulation function modified to collect data for animation in Colab
    """
    # Create data containers
    goal = [cx[-1], cy[-1]]
    state = initial_state

    # Initial yaw compensation
    if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
    elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0

    time = 0.0
    x = [state.x]
    y = [state.y]
    yaw = [state.yaw]
    v = [state.v]
    t = [0.0]
    d = [0.0]
    a = [0.0]
    target_ind, _ = calc_nearest_index(state, cx, cy, cyaw, 0)

    odelta, oa = None, None
    cyaw = smooth_yaw(cyaw)

    # Initialize animation data collection
    animation_data = [] if show_animation else None

    # Simulation loop
    try:
        while MAX_TIME >= time:
            # Generate reference trajectory
            xref, target_ind, dref = calc_ref_trajectory(
                state, cx, cy, cyaw, ck, sp, dl, target_ind)

            # Current state
            x0 = [state.x, state.y, state.v, state.yaw]

            # MPC control
            oa, odelta, ox, oy, oyaw, ov = iterative_linear_mpc_control(
                xref, x0, dref, oa, odelta)

            # Apply first control input
            di, ai = 0.0, 0.0
            if odelta is not None:
                di, ai = odelta[0], oa[0]
                state = update_state(state, ai, di)

            # Update time and store data
            time = time + DT
            x.append(state.x)
            y.append(state.y)
            yaw.append(state.yaw)
            v.append(state.v)
            t.append(time)
            d.append(di)
            a.append(ai)

            # Collect animation data
            if show_animation:
                current_data = {
                    'x': state.x,
                    'y': state.y,
                    'yaw': state.yaw,
                    'v': state.v,
                    'steer': di,
                    'xref': xref.copy(),
                    'ox': ox.copy() if ox is not None else None,
                    'oy': oy.copy() if oy is not None else None,
                    'target_ind': target_ind,
                    'time': time
                }
                animation_data.append(current_data)

            # Check goal
            if check_goal(state, goal, target_ind, len(cx)):
                print("Goal")
                break
    except Exception as e:
        print(f"Simulation error: {e}")

    return t, x, y, yaw, v, d, a, animation_data


## Function to Run MPC Simulation in Offline Mode

In [ ]:
def run_mpc_simulation(trajectory_name, target_speed_kmh=10.0, dl=1.0, animation=True):
    """Run MPC simulation with selected trajectory and inline animation in Colab"""
    # Convert km/h to m/s
    target_speed = target_speed_kmh / 3.6

    # Get waypoints
    waypoints = TRAJECTORIES[trajectory_name]()

    # Create trajectory
    cx, cy, cyaw, ck = create_custom_trajectory(waypoints, dl)

    # Calculate speed profile
    sp = calc_speed_profile(cx, cy, cyaw, target_speed)

    # Set initial state
    initial_state = State(x=cx[0], y=cy[0], yaw=cyaw[0], v=0.0)

    # Print info
    print(f"Running simulation for '{trajectory_name}' trajectory")
    print(f"Target speed: {target_speed_kmh} km/h")

    # Run simulation
    t, x, y, yaw, v, d, a, animation_data = do_simulation(
        cx, cy, cyaw, ck, sp, dl, initial_state, show_animation=animation)

    # Display animation if enabled
    if animation and animation_data is not None:
        fig, ax = plt.subplots(figsize=(10, 8))

        # Draw the entire trajectory first
        ax.plot(cx, cy, "-r", label="course")
        ax.plot(x, y, "-b", label="trajectory")
        ax.axis("equal")
        ax.grid(True)
        ax.legend()
        plt.show()

        # Now create an animation that will save to HTML
        fig, ax = plt.subplots(figsize=(10, 8))

        def update_frame(i):
            ax.clear()
            current_data = animation_data[i]

            # Plot reference trajectory
            ax.plot(cx, cy, "-r", label="course")
            # Plot vehicle trajectory up to current time
            ax.plot([d['x'] for d in animation_data[:i+1]],
                    [d['y'] for d in animation_data[:i+1]], "-b", label="trajectory")
            # Plot MPC prediction
            if current_data['ox'] is not None:
                ax.plot(current_data['ox'], current_data['oy'], "xr", label="MPC")
            # Plot reference points
            ax.plot(current_data['xref'][0, :], current_data['xref'][1, :], "xk", label="xref")
            # Plot target point
            ax.plot(cx[current_data['target_ind']], cy[current_data['target_ind']], "xg", label="target")
            # Plot vehicle
            plot_car(current_data['x'], current_data['y'], current_data['yaw'],
                     steer=current_data['steer'], ax=ax)
            ax.axis("equal")
            ax.grid(True)
            ax.set_title(f"Time: {current_data['time']:.2f} s, Speed: {current_data['v']*3.6:.2f} km/h")
            if i == 0:  # Add legend only on first frame to avoid clutter
                ax.legend()

        # Use a sampling approach for longer animations to keep the file size reasonable
        total_frames = len(animation_data)

        if total_frames > 100:
            # For longer simulations, sample frames to get approximately 100 frames
            # This ensures we get the full trajectory but keeps file size reasonable
            sample_rate = max(1, total_frames // 100)
            sampled_frames = list(range(0, total_frames, sample_rate))
            # Always include the last frame to show completion
            if (total_frames - 1) not in sampled_frames:
                sampled_frames.append(total_frames - 1)

            # Create a function to get the animation data at the sampled indices
            def get_frame(i):
                return update_frame(sampled_frames[i])

            ani = FuncAnimation(fig, get_frame, frames=len(sampled_frames), interval=100)
        else:
            # For shorter simulations, show all frames
            ani = FuncAnimation(fig, update_frame, frames=total_frames, interval=100)

        plt.close()  # Prevents duplicate display
        display(HTML(ani.to_jshtml()))

    # Plot result summary
    plt.figure(figsize=(12, 9))
    plt.subplot(3, 1, 1)
    plt.plot(cx, cy, "-r", label="reference")
    plt.plot(x, y, "-b", label="trajectory")
    plt.grid(True)
    plt.axis("equal")
    plt.legend()

    plt.subplot(3, 1, 2)
    plt.plot(t, [speed * 3.6 for speed in v], "-r", label="speed")
    plt.grid(True)
    plt.xlabel("Time [s]")
    plt.ylabel("Speed [km/h]")

    plt.subplot(3, 1, 3)
    plt.plot(t, [np.rad2deg(steer) for steer in d], "-r", label="steering")
    plt.grid(True)
    plt.xlabel("Time [s]")
    plt.ylabel("Steering [deg]")

    plt.tight_layout()
    plt.show()

    return t, x, y, yaw, v, d, a

# Real Time simulation function

In [ ]:
def run_live_simulation(trajectory_name="Circular", target_speed_kmh=10.0, dl=1.0):
    """
    Run MPC simulation with live updates as calculations are performed
    """
    from IPython.display import clear_output
    import time as real_time

    # Convert km/h to m/s
    target_speed = target_speed_kmh / 3.6

    # Get waypoints
    waypoints = TRAJECTORIES[trajectory_name]()

    # Create trajectory
    cx, cy, cyaw, ck = create_custom_trajectory(waypoints, dl)

    # Calculate speed profile
    sp = calc_speed_profile(cx, cy, cyaw, target_speed)

    # Set initial state
    initial_state = State(x=cx[0], y=cy[0], yaw=cyaw[0], v=0.0)

    # Print info
    print(f"Running live simulation for '{trajectory_name}' trajectory")
    print(f"Target speed: {target_speed_kmh} km/h")

    # Initialize variables for simulation
    goal = [cx[-1], cy[-1]]
    state = initial_state

    # Initial yaw compensation
    if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
    elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0

    simulation_time = 0.0
    history_x = [state.x]
    history_y = [state.y]
    history_yaw = [state.yaw]
    history_v = [state.v]
    history_t = [0.0]
    history_d = [0.0]
    history_a = [0.0]

    target_ind, _ = calc_nearest_index(state, cx, cy, cyaw, 0)
    odelta, oa = None, None
    cyaw = smooth_yaw(cyaw)

    # Get axis limits for consistent plotting
    x_min, x_max = min(cx)-5, max(cx)+5
    y_min, y_max = min(cy)-5, max(cy)+5

    # Simulation loop
    try:
        while MAX_TIME >= simulation_time:
            # Create real-time plot
            plt.figure(figsize=(10, 8))

            # Plot reference path
            plt.plot(cx, cy, "-r", label="reference")

            # Plot history of vehicle positions
            plt.plot(history_x, history_y, "-b", label="trajectory")

            # Generate reference trajectory
            xref, target_ind, dref = calc_ref_trajectory(
                state, cx, cy, cyaw, ck, sp, dl, target_ind)

            # Plot reference points
            plt.plot(xref[0, :], xref[1, :], "xk", label="xref")

            # Plot target point
            plt.plot(cx[target_ind], cy[target_ind], "xg", label="target")

            # Current state
            x0 = [state.x, state.y, state.v, state.yaw]

            # MPC control
            oa, odelta, ox, oy, oyaw, ov = iterative_linear_mpc_control(
                xref, x0, dref, oa, odelta)

            # Plot MPC prediction if available
            if ox is not None:
                plt.plot(ox, oy, "xr", label="MPC")

            # Plot vehicle
            plot_car(state.x, state.y, state.yaw, steer=odelta[0] if odelta is not None else 0)

            # Set consistent axis limits
            plt.xlim(x_min, x_max)
            plt.ylim(y_min, y_max)
            plt.axis("equal")
            plt.grid(True)
            plt.title(f"Time: {simulation_time:.2f}s, Speed: {state.v*3.6:.2f} km/h")
            plt.legend()

            # Display the plot and clear for the next iteration
            display(plt.gcf())
            clear_output(wait=True)
            plt.close()

            # Apply control
            di, ai = 0.0, 0.0
            if odelta is not None:
                di, ai = odelta[0], oa[0]

            # Update state
            state = update_state(state, ai, di)

            # Update time and store history
            simulation_time = simulation_time + DT
            history_x.append(state.x)
            history_y.append(state.y)
            history_yaw.append(state.yaw)
            history_v.append(state.v)
            history_t.append(simulation_time)
            history_d.append(di)
            history_a.append(ai)

            # Real-time delay to slow down visualization
            real_time.sleep(0.005)  # Adjust this value to control display speed

            # Check goal
            if check_goal(state, goal, target_ind, len(cx)):
                print("Goal reached!")
                break

    except Exception as e:
        print(f"Simulation error: {e}")

    # Final display of results
    plt.figure(figsize=(12, 9))
    plt.subplot(3, 1, 1)
    plt.plot(cx, cy, "-r", label="reference")
    plt.plot(history_x, history_y, "-b", label="trajectory")
    plt.grid(True)
    plt.axis("equal")
    plt.legend()

    plt.subplot(3, 1, 2)
    plt.plot(history_t, [speed * 3.6 for speed in history_v], "-r", label="speed")
    plt.grid(True)
    plt.xlabel("Time [s]")
    plt.ylabel("Speed [km/h]")

    plt.subplot(3, 1, 3)
    plt.plot(history_t, [np.rad2deg(steer) for steer in history_d], "-r", label="steering")
    plt.grid(True)
    plt.xlabel("Time [s]")
    plt.ylabel("Steering [deg]")

    plt.tight_layout()
    plt.show()

    return history_t, history_x, history_y, history_yaw, history_v, history_d, history_a

# RUN

## Offline Mode with animation pane (wait untill the cell run is completed)

In [ ]:
# Run simulation with popup window
trajectory_name = "Circular"
target_speed = 10.0  # km/h
dl = 1.0

_ = run_mpc_simulation(trajectory_name, target_speed, dl, animation=True)

## Realtime Simulation (Slower)

In [ ]:
# Run live simulation
_ = run_live_simulation(trajectory_name="Circular", target_speed_kmh=10.0)